In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer

2023-04-02 17:23:42.874759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 17:23:43.661410: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/kacper/Code/AnalizaSympatii/AnalizaSympatiiEnv/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuratio

In [2]:
np.random.seed(42)

In [3]:
raw_dataset = pd.read_csv("data/file.csv", index_col=0).to_numpy()

In [4]:
np.random.shuffle(raw_dataset)

In [5]:
WHOLE_DATASET_LEN = 20000
lb = LabelBinarizer()
X, y  = raw_dataset[:, 0], lb.fit_transform(raw_dataset[:, 1])
X, y  = X[:WHOLE_DATASET_LEN], y[:WHOLE_DATASET_LEN]

In [6]:
X_train, X_test = X[:int(WHOLE_DATASET_LEN*0.7)], X[int(WHOLE_DATASET_LEN*0.7):]
y_train, y_test = y[:int(WHOLE_DATASET_LEN*0.7)], y[int(WHOLE_DATASET_LEN*0.7):]

In [7]:
test_set_len = int(len(X_test) * 0.7)
X_test, X_valid = X_test[:test_set_len], X_test[test_set_len:]
y_test, y_valid = y_test[:test_set_len], y_test[test_set_len:]

# BERT

In [8]:
#tiny bert l=2 h=128
tiny_bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2"
bert_preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [9]:
def build_bert_model(preprocessor_url, model_url):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(bert_preprocessor_url, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tiny_bert_model_url, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(3, activation="softmax", name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [10]:
tiny_bert_model = build_bert_model(bert_preprocessor_url, tiny_bert_model_url)

2023-04-02 17:23:47.409783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-02 17:23:47.468926: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-04-02 17:23:48.202543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq

In [11]:
loss = tf.keras.losses.CategoricalCrossentropy()
metric = tf.keras.metrics.CategoricalCrossentropy()

In [12]:
epochs = 8 
steps_pre_epoch = len(X_train) 
num_train_steps = steps_pre_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(
    init_lr = init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type='adamw'
)

In [13]:
tiny_bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric, 'accuracy'])

In [14]:
tiny_bert_model_history = tiny_bert_model.fit(x=X_train, y=y_train,  validation_data=(X_valid, y_valid), epochs=epochs)

Epoch 1/8


2023-04-02 17:23:53.389949: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_1' with dtype int32 and shape [?,128]
	 [[{{node inputs_1}}]]
2023-04-02 17:23:53.390075: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_2' with dtype int32 and shape [?,128]
	 [[{{node inputs_2}}]]
2023-04-02 17:23:54.016594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/Stateful

438/438 [==============================] - 80s 177ms/step - loss: 1.4940 - categorical_crossentropy: 1.4940 - accuracy: 0.2628 - val_loss: 1.2150 - val_categorical_crossentropy: 1.2150 - val_accuracy: 0.2839
Epoch 2/8
438/438 [==============================] - 76s 174ms/step - loss: 1.1477 - categorical_crossentropy: 1.1477 - accuracy: 0.3849 - val_loss: 1.0266 - val_categorical_crossentropy: 1.0266 - val_accuracy: 0.4900
Epoch 3/8
438/438 [==============================] - 76s 174ms/step - loss: 1.0579 - categorical_crossentropy: 1.0579 - accuracy: 0.4739 - val_loss: 0.9803 - val_categorical_crossentropy: 0.9803 - val_accuracy: 0.5306
Epoch 4/8
438/438 [==============================] - 77s 177ms/step - loss: 1.0249 - categorical_crossentropy: 1.0249 - accuracy: 0.5056 - val_loss: 0.9572 - val_categorical_crossentropy: 0.9572 - val_accuracy: 0.5406
Epoch 5/8
438/438 [==============================] - 79s 182ms/step - loss: 0.9927 - categorical_crossentropy: 0.9927 - accuracy: 0.5236 -

KeyboardInterrupt: 

In [ ]:
tiny_bert_model.evaluate(X_test,y_test)

132/132 [==============================] - 5s 41ms/step - loss: 0.8046 - categorical_crossentropy: 0.8046 - accuracy: 0.6350


[0.8046244382858276, 0.8046244382858276, 0.6349999904632568]

In [ ]:
mini_bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2"

In [ ]:
mini_bert_model = build_bert_model(bert_preprocessor_url, tiny_bert_model_url)


In [ ]:
mini_bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric, 'accuracy'])

In [ ]:
mini_bert_model_history = mini_bert_model.fit(x=X_train, y=y_train,  validation_data=(X_valid, y_valid), epochs=epochs)

Epoch 1/8
438/438 [==============================] - 70s 155ms/step - loss: 1.0107 - categorical_crossentropy: 0.9631 - accuracy: 0.5151 - val_loss: 0.8908 - val_categorical_crossentropy: 0.8908 - val_accuracy: 0.5850
Epoch 2/8
438/438 [==============================] - 68s 156ms/step - loss: 0.8729 - categorical_crossentropy: 0.8729 - accuracy: 0.5975 - val_loss: 0.7948 - val_categorical_crossentropy: 0.7948 - val_accuracy: 0.6594
Epoch 3/8
438/438 [==============================] - 68s 156ms/step - loss: 0.7885 - categorical_crossentropy: 0.7885 - accuracy: 0.6434 - val_loss: 0.7072 - val_categorical_crossentropy: 0.7072 - val_accuracy: 0.6789
Epoch 4/8
438/438 [==============================] - 66s 151ms/step - loss: 0.7240 - categorical_crossentropy: 0.7240 - accuracy: 0.6820 - val_loss: 0.6733 - val_categorical_crossentropy: 0.6733 - val_accuracy: 0.7028
Epoch 5/8
438/438 [==============================] - 65s 147ms/step - loss: 0.6782 - categorical_crossentropy: 0.6782 - accuracy

In [ ]:
mini_bert_model.evaluate(X_test,y_test)

132/132 [==============================] - 5s 41ms/step - loss: 0.5647 - categorical_crossentropy: 0.5647 - accuracy: 0.7729


[0.564741849899292, 0.564741849899292, 0.772857129573822]

In [ ]:
medium_bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2"

In [ ]:

medium_bert_model = build_bert_model(bert_preprocessor_url, tiny_bert_model_url)

In [ ]:

medium_bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric, 'accuracy'])


In [ ]:
medium_bert_model_history = mini_bert_model.fit(x=X_train, y=y_train,  validation_data=(X_valid, y_valid), epochs=epochs)

Epoch 1/8
438/438 [==============================] - 66s 151ms/step - loss: 0.5295 - categorical_crossentropy: 0.5295 - accuracy: 0.7829 - val_loss: 0.5404 - val_categorical_crossentropy: 0.5404 - val_accuracy: 0.7828
Epoch 2/8
438/438 [==============================] - 67s 153ms/step - loss: 0.4948 - categorical_crossentropy: 0.4948 - accuracy: 0.8014 - val_loss: 0.5317 - val_categorical_crossentropy: 0.5317 - val_accuracy: 0.7967
Epoch 3/8
438/438 [==============================] - 66s 151ms/step - loss: 0.4695 - categorical_crossentropy: 0.4695 - accuracy: 0.8144 - val_loss: 0.5192 - val_categorical_crossentropy: 0.5192 - val_accuracy: 0.7994
Epoch 4/8
438/438 [==============================] - 67s 152ms/step - loss: 0.4445 - categorical_crossentropy: 0.4445 - accuracy: 0.8224 - val_loss: 0.5132 - val_categorical_crossentropy: 0.5132 - val_accuracy: 0.8111
Epoch 5/8
438/438 [==============================] - 67s 153ms/step - loss: 0.4216 - categorical_crossentropy: 0.4216 - accuracy

In [ ]:
medium_bert_model.evaluate(X_test,y_test)

132/132 [==============================] - 6s 42ms/step - loss: 1.1147 - categorical_crossentropy: 1.1147 - accuracy: 0.3960


[1.1146581172943115, 1.1146581172943115, 0.39595237374305725]

# Biblio
https://www.tensorflow.org/text/tutorials/classify_text_with_bert